# raw_data.py

In [15]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import psycopg2

conn = psycopg2.connect(database="solar",
						user="ai_guy",
						password="AcBel.26217672",
						host="10.10.171.136",
						port="52102")

In [5]:
sql_param = \
'''
SELECT data_time, value_1 AS irradiance 
FROM sl42_irradiance sl42
WHERE site_id = %(site_id)s
	AND data_time BETWEEN %(startDT)s AND %(endDT)s
ORDER BY data_time ASC
'''

In [6]:
site_id = 77
year = 2021
month = 9
date = 1

df_db = pd.read_sql_query(	sql_param,
							con = conn,
							params={'site_id':site_id,
									'startDT':datetime(year, month, date, 0, 0, 0),
									'endDT':datetime(year, month, date, 23, 59, 59)},
							parse_dates = {'data_time':{'format':'%Y-%m-%d %H:%M:%S:%f'}},
							index_col = ['data_time'])

df_db = df_db.dropna(axis = 'columns', how = 'all')
df_db

,irradiance
data_time,
2021-09-01 05:00:00+08:00,0.0
2021-09-01 05:02:00+08:00,0.0
2021-09-01 05:04:00+08:00,0.0
2021-09-01 05:06:00+08:00,0.0
2021-09-01 05:08:00+08:00,0.0
...,...
2021-09-01 19:50:00+08:00,0.0
2021-09-01 19:52:00+08:00,0.0
2021-09-01 19:54:00+08:00,0.0


# train_data.py

In [20]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)


time_steps = 3
feature_dim = 1

X, y = split_sequence(  df_db['irradiance'].values,
                        time_steps)

X = X.reshape(  X.shape[0],
                X.shape[1],
                feature_dim)

np.savez(   'train_data.npz',
            x=X,
            y=y)

# model.py

In [18]:
input = tf.keras.Input(shape=(  time_steps,
                                feature_dim))

x = tf.keras.layers.Conv1D(filters=64,
                           kernel_size=2,
                           padding='valid',
                           activation='relu',
                           strides=1)(input)
x = tf.keras.layers.MaxPooling1D(pool_size=2)(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(50, 
                          activation='relu')(x)
output = tf.keras.layers.Dense(1, 
                               activation='relu')(x)

model = tf.keras.Model(inputs=input, 
                       outputs=output)

model.compile(loss='mse',
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3, 1)]            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 3,493
Trainable params: 3,493
Non-trainable params: 0
_________________________________________________________

In [22]:
print('training model...')

training model...
